**<font color = black size=6>实验五:随机森林</font>**

本次实验为编写集成学习中的随机森林算法。在上一次实验中，我们已经学会了如何构建一棵ID3决策树。在本次实验，我们将以上一次决策树代码的基础上，结合集成学习中的并行化生成分类模型的思想，构建多棵决策树，组成随机森林。

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
# tem = set(range(5))
# print(len(tem))
# # tem = list(tem)
# # tem
# a = np.random.choice(list(tem), replace=True, size=10)
# for i in a:
#     print(i)
# a = np.array([1,2,3])
b = np.array([3,4,5])
b[-1]
# temlist = []
# temlist.append(a)
# temlist.append(b)
# temlist = np.array(temlist)
# temlist[:,0]
# int(1.0)

5

**<font color = blue size=4>第一部分:函数介绍</font>**

介绍一些在数据采样和属性集采样的过程中可以用到的随机函数。

In [19]:
# np.random.choice函数从一个一维数组中随机采样
x = np.array([1,2,3,4])
y = np.random.choice(x, replace=True, size=10)
print(y)

# np.random.shuffle函数对一个数组/矩阵按照第一维进行洗牌
x = np.array([[0,1,2],[3,4,5],[6,7,8],[9,10,11],[12,13,14]])
np.random.shuffle(x)
print(x)

# DataFrame对象的sample函数可以随机采样n个数据或者采样比例为frac的数据
x = np.array([[0,0,0],[1,1,1],[2,2,2],[3,3,3],[4,4,4]])
frame = pd.DataFrame(x)
print(frame.sample(n=2))
print(frame.sample(frac=0.3))

ValueError: Cannot take a larger sample than population when 'replace=False'

**<font color = blue size=4>第二部分:实验任务</font>**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from collections import Counter

本次实验承接上次实验，实现随机森林。

<span style="color:purple">1) 采用自助采样法对训练数据集'train_titanic.csv'进行采样，生成$n$个训练数据集($n$自行设定)。自助采样法是指，每次从原本数据集中【有放回】地随机采样一个数据，重复进行$m$次，就生成一个有$m$个数据的训练数据集($m$是原本数据集的数据个数)。</span>

In [24]:
train_frame = pd.read_csv('train_titanic.csv')

# Bootstrap 采样
n = 10
m = len(train_frame)
new_train = []

for i in range(n):
    tem = train_frame.sample(1)
    for j in range(1,m):
        tem = tem.append(train_frame.sample(1))      
    new_train.append(tem)


<span style="color:purple">2) 对上次实验的best_split函数进行修改，改成先从属性集$A$中先随机选取$k$个属性构成属性集$A'$，再从$A'$中选取最佳划分的属性。($k$是一个整数，一般取$max(round(log_2 d),1)$, 其中$d$是$A$的元素的个数)</span>

In [25]:
def entropy(label):
    label = label.reshape(len(label),1)
    counter = Counter(label[:,0])
    a=np.unique(label[:,0])
    ent=0
    m = len(label)
    for i in range(len(a)):
        ent -= counter[a[i]]/m*np.log2(counter[a[i]]/m)
    return ent

def split(feature, label, dimension):
    label = label.reshape(len(label),1)
    a=np.unique(feature[:,dimension])
    split_feature = []
    split_label = []
    for i in range(len(a)):
        split_feature.append([])
        split_label.append([])
    for i in range(len(label)):
        for j in range(len(a)):
            if feature[i,dimension]==a[j]:
                split_feature[j].append(feature[i,:])
                split_label[j].append(label[i,:])
    for i in range(len(a)):
        split_feature[i] = np.array(split_feature[i])
        split_label[i] = np.array(split_label[i])
    split_feature = np.array(split_feature)
    split_label = np.array(split_label)
    return split_feature,split_label

def best_split(D, A):
    best_entropy = -100
    best_dimension = -1
    ladi = D.shape[1]-1
    tot = entropy(D[:,ladi])
    for i in A:
        tem = tot
        split_feature,split_label=split(D,D[:,D.shape[1]-1],i)
        for j in range(len(split_label)):
            tem-=len(split_label[j])/len(D)*entropy(split_label[j])
        if tem>best_entropy:
            best_entropy=tem
            best_dimension=i
    return best_dimension

def new_best_split(D,A):
    k = int(max(round(np.log2(len(A))),1))
    A_new = np.random.choice(list(A), replace=False, size=k)
    result = best_split(D,A_new)
    return result

def same(D,A):
    tem = np.sum(D,axis=0)/len(D)
    for i in A:
        if all(D[:,i] != tem[i]):
            return False
    return True

<span style="color:purple">3) 对上次实验完成的决策树类进行如下修改：①predict函数不需要计算预测准确率，要返回数据集D的预测标签；②每个属性的可能取值possible_value不从采样的数据集中取，而是从原本数据集'train_titanic.csv'中取，以防止在预测过程中出现决策树在训练过程中未见过的属性取值。</span>

In [26]:
# 记下所有属性可能的取值
D = np.array(train_frame)
A = set(range(D.shape[1] - 1))
possible_value = {}
for every in A:
    possible_value[every] = np.unique(D[:, every])

In [27]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.index = index # index表示该分支结点的划分属性的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分属性的值为val的孩子结点

In [28]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = possible_value # 用于存储每个属性可能的取值
    
        
    '''
    TreeGenerate函数用于递归构建决策树，伪代码参照课件中的“Algorithm 1 决策树学习基本算法”
    '''
    def TreeGenerate(self, D, A):
        
        # 生成结点 node
        node = Node()
        
        
        
        # if D中样本全属于同一类别C then
        #     将node标记为C类叶结点并返回
        # end if
        ladi = D.shape[1]-1
        counter = Counter(D[:,ladi])
        if counter[0] == len(D):
            node.label = 0
            node.isLeaf = True
            return node
        elif counter[1] == len(D):
            node.label = 1
            node.isLeaf = True 
            return node
        
        
        
        # if A = Ø OR D中样本在A上取值相同 then
        #     将node标记叶结点，其类别标记为D中样本数最多的类并返回
        # end if
        if (len(A)==0) or same(D,A):
            if counter[0]>=counter[1]:
                node.label = 0
            else:
                node.label = 1
            node.isLeaf = True
            return node
        
        
        
        
        # 从A中选择最优划分属性a_star
        # （选择信息增益最大的属性，用到上面实现的best_split函数）
        a_star = new_best_split(D, A)
        
        
        
        # for a_star 的每一个值a_star_v do
        #     为node 生成每一个分支；令D_v表示D中在a_star上取值为a_star_v的样本子集
        #     if D_v 为空 then
        #         将分支结点标记为叶结点，其类别标记为D中样本最多的类
        #     else
        #         以TreeGenerate(D_v,A-{a_star}) 为分支结点
        #     end if
        # end for
        tem = Counter(D[:,a_star])

        node.isLeaf = False
        node.index = a_star

        for a_star_v in self.possible_value[a_star]:
            D_v = []
            newnode = Node()                       
            for i in range(len(D)):
                if D[i,a_star] == a_star_v:
                    D_v.append(D[i,:])
            D_v = np.array(D_v)
            
            if(len(D_v)==0):
                newnode.label = 0
                newnode.isleaf = True
                           ############
            else:
                newnode = self.TreeGenerate(D_v,A-{a_star})
            node.addNode(a_star_v,newnode)
    
        return node
    
    
    
    
    '''
    train函数可以做一些数据预处理（比如Dataframe到numpy矩阵的转换，提取属性集等），并调用TreeGenerate函数来递归地生成决策树
    '''
    def train(self, D):
        D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        A = set(range(D.shape[1] - 1)) # 属性集A
        self.tree_root = self.TreeGenerate(D, A) # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root

    
    
    
    
    '''
    predict函数对测试集D进行预测，输出预测标签
    '''
    def predict(self, D):
        D = np.array(D) # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        
#         #对于D中的每一行数据d，从当前结点x=self.tree_root开始，当当前结点x为分支结点时，
#         #则搜索x的划分属性为该行数据相应的属性值的孩子结点（即x=x.children[d[x.index]]），不断重复，
#         #直至搜索到叶结点，该叶结点的label就是数据d的预测label
        result = []
        for i in range(len(D)):
            x = self.tree_root
            d = D[i,:]
            while x.isLeaf == False:

                x=x.children[d[x.index]]
            result.append(x.label)
        result = np.array(result)
        return result
        

<span style="color:purple">4) 生成$n$棵决策树实例，每棵决策树对上面生成的$n$个训练数据集中的一个数据集进行训练。</span>

In [29]:
# ----- Your code here -------
dtree = []
for i in range(n):
    new_tree = DTree()
    new_tree.train(new_train[i])
    dtree.append(new_tree)




<span style="color:purple">5) 用训练完成的$n$棵决策树分别对测试数据集'test_titanic.csv'进行预测。采用相对多数投票法$H(x)=C_{\mathop{\arg\max}_{j} \sum_{i=1}^T h_i^j(x)}$来对各棵决策树的预测结果进行结合。输出结合的预测结果的准确率。</span>

In [30]:
test_frame = pd.read_csv('test_titanic.csv')

# ----- Your code here -------

test = np.array(test_frame)
totallabel = []
for i in range(n):
    totallabel.append(dtree[i].predict(test_frame))
totallabel = np.array(totallabel)
correct = 0
for i in range(len(test)):
    counter1 = Counter(totallabel[:,i])
    if counter1[1] >= counter1[0]:
        if test[i,test.shape[1]-1] == 1:
            correct += 1
    else:
        if test[i,test.shape[1]-1] == 0:
            correct += 1
correct/len(test)


0.8415841584158416

**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、因为下周放假冲了一次理论课，本次实验做两周，实验报告下下周（4月15号前）交  
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验五(随机森林)的实验报告上交地址:https://workspace.jianguoyun.com/inbox/collect/3c4acbb1e9a044c48fec14e2fdb97b56/submit

三、课堂课件获取地址:https://www.jianguoyun.com/p/DQlpUFYQp5WhChiS_q0E  
实验内容获取地址:https://www.jianguoyun.com/p/DbKbP-AQp5WhChi1sa0E